# Initialisation


In [ ]:
!pip install unidecode

# Import required packages
from transformers import pipeline, DataCollatorWithPadding
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from google.colab import files
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from google.colab import files
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import pandas as pd
import numpy as np
import torch
import xgboost as xgb
import torchgen
import string
import unidecode
import nltk
import torch
import xgboost as xgb


torch.cuda.is_available()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.6 MB/s eta 0:00:00


False

In [ ]:
# URL to the raw CSV file on GitHub
url = 'https://raw.githubusercontent.com/msperand/Machine_Learning_Project/main/Data/sample_submission.csv'

# Read the CSV file into a DataFrame
sample_submission = pd.read_csv(url)

# Display the first few rows of the DataFrame
print(sample_submission)


        id difficulty
0        0         A1
1        1         A1
2        2         A1
3        3         A1
4        4         A1
...    ...        ...
1195  1195         A1
1196  1196         A1
1197  1197         A1
1198  1198         A1
1199  1199         A1

[1200 rows x 2 columns]


In [ ]:
# URL to the raw CSV file on GitHub
url2 = 'https://raw.github.com/msperand/Machine_Learning_Project/main/Data/training_data.csv'

# Read the CSV file into a DataFrame
training_data = pd.read_csv(url2)

# Display the first few rows of the DataFrame
print(training_data)

        id                                           sentence difficulty
0        0  Les coûts kilométriques réels peuvent diverger...         C1
1        1  Le bleu, c'est ma couleur préférée mais je n'a...         A1
2        2  Le test de niveau en français est sur le site ...         A1
3        3           Est-ce que ton mari est aussi de Boston?         A1
4        4  Dans les écoles de commerce, dans les couloirs...         B1
...    ...                                                ...        ...
4795  4795  C'est pourquoi, il décida de remplacer les hab...         B2
4796  4796  Il avait une de ces pâleurs splendides qui don...         C1
4797  4797  Et le premier samedi de chaque mois, venez ren...         A2
4798  4798  Les coûts liés à la journalisation n'étant pas...         C2
4799  4799  Sur le sable, la mer haletait de toute la resp...         C2

[4800 rows x 3 columns]


In [ ]:
# URL to the raw CSV file on GitHub
url3 = 'https://raw.github.com/msperand/Machine_Learning_Project/main/Data/unlabelled_test_data.csv'

# Read the CSV file into a DataFrame
test_data = pd.read_csv(url3)

# Display the first few rows of the DataFrame
print(test_data)

        id                                           sentence
0        0  Nous dûmes nous excuser des propos que nous eû...
1        1  Vous ne pouvez pas savoir le plaisir que j'ai ...
2        2  Et, paradoxalement, boire froid n'est pas la b...
3        3  Ce n'est pas étonnant, car c'est une saison my...
4        4  Le corps de Golo lui-même, d'une essence aussi...
...    ...                                                ...
1195  1195  C'est un phénomène qui trouve une accélération...
1196  1196  Je vais parler au serveur et voir si on peut d...
1197  1197  Il n'était pas comme tant de gens qui par pare...
1198  1198      Ils deviennent dangereux pour notre économie.
1199  1199  Son succès a généré beaucoup de réactions néga...

[1200 rows x 2 columns]


# Knowing the data




In [ ]:

# Assuming 'test_data' contains a column named 'sentence' which holds the text data
# Calculate the length of each sentence
test_data['sentence_length'] = test_data['sentence'].apply(len)

# Display summary statistics for the sentence lengths
sentence_length_stats = test_data['sentence_length'].describe()

# Print the summary statistics
print("Summary Statistics of Sentence Lengths:\n", sentence_length_stats)


Summary Statistics of Sentence Lengths:
 count    1200.000000
mean      109.152500
std        96.488078
min         6.000000
25%        49.000000
50%        81.000000
75%       144.000000
max       930.000000
Name: sentence_length, dtype: float64


In [ ]:
# Calculate the length of each sentence
training_data['sentence_length'] = training_data['sentence'].apply(len)

# Display summary statistics for the sentence lengths
sentence_length_stats = training_data['sentence_length'].describe()

# Print the summary statistics
print("Summary Statistics of Sentence Lengths:\n", sentence_length_stats)


Summary Statistics of Sentence Lengths:
 count    4800.000000
mean      110.355208
std       100.873552
min         4.000000
25%        49.000000
50%        84.000000
75%       142.000000
max      1572.000000
Name: sentence_length, dtype: float64


In [ ]:
# Count the number of characters in each sentence
training_data['char_count'] = training_data['sentence'].apply(len)

# Filter out sentences with more than 1000 characters
original_count = len(training_data)  # Store the original number of sentences for comparison
training_data = training_data[training_data['char_count'] <= 1000]

# Drop the 'char_count' column as it is no longer needed
training_data.drop(columns=['char_count'], inplace=True)

# Display the first few rows of the updated DataFrame to ensure it's processed correctly
print("Updated training_data with sentences of 1000 characters or less:")
print(training_data.head())

# Display the number of sentences removed
print("\nNumber of sentences removed:", original_count - len(training_data))


Updated training_data with sentences of 1000 characters or less:
   id                                           sentence difficulty  \
0   0  Les coûts kilométriques réels peuvent diverger...         C1   
1   1  Le bleu, c'est ma couleur préférée mais je n'a...         A1   
2   2  Le test de niveau en français est sur le site ...         A1   
3   3           Est-ce que ton mari est aussi de Boston?         A1   
4   4  Dans les écoles de commerce, dans les couloirs...         B1   

   sentence_length  
0              255  
1               62  
2               66  
3               40  
4              209  

Number of sentences removed: 5


<ipython-input-7-ade5356ceb63>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data.drop(columns=['char_count'], inplace=True)


# Dummy tries

In [ ]:
# Assuming 'sample_submission' is your DataFrame
sample_submission['difficulty'] = 'A2'

# Now the 'difficulty' column should be updated to 'A2' for all rows
# Display the updated DataFrame to confirm the changes
print(sample_submission)


        id difficulty
0        0         A2
1        1         A2
2        2         A2
3        3         A2
4        4         A2
...    ...        ...
1195  1195         A2
1196  1196         A2
1197  1197         A2
1198  1198         A2
1199  1199         A2

[1200 rows x 2 columns]


In [ ]:
# Save the updated DataFrame to a CSV file
file_name = 'sample_submission_A2.csv'  # Temporary file name in the Colab environment
sample_submission.to_csv(file_name, index=False)

In [ ]:
# Assuming 'sample_submission' is your DataFrame which has been updated
sample_submission['difficulty'] = 'B1'  # Updating the difficulty column to 'A2'

# Save the updated DataFrame to a CSV file
file_name = 'sample_submission_B1.csv'  # Temporary file name in the Colab environment
sample_submission.to_csv(file_name, index=False)

In [ ]:
# Assuming 'sample_submission' is your DataFrame which has been updated
sample_submission['difficulty'] = 'B2'  # Updating the difficulty column to 'A2'

# Save the updated DataFrame to a CSV file
file_name = 'sample_submission_B2.csv'  # Temporary file name in the Colab environment
sample_submission.to_csv(file_name, index=False)

In [ ]:
# Assuming 'sample_submission' is your DataFrame which has been updated
sample_submission['difficulty'] = 'C1'  # Updating the difficulty column to 'A2'

# Save the updated DataFrame to a CSV file
file_name = 'sample_submission_C1.csv'  # Temporary file name in the Colab environment
sample_submission.to_csv(file_name, index=False)

In [ ]:
# Assuming 'sample_submission' is your DataFrame which has been updated
sample_submission['difficulty'] = 'C2'  # Updating the difficulty column to 'A2'

# Save the updated DataFrame to a CSV file
file_name = 'sample_submission_C2.csv'  # Temporary file name in the Colab environment
sample_submission.to_csv(file_name, index=False)

# Basic tries

In [ ]:
# Split the training data
X_train = training_data['sentence']
y_train = training_data['difficulty']

# Create a TF-IDF vectorizer and Naive Bayes classifier pipeline
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Train the model
model.fit(X_train, y_train)

# Predict the difficulty for the test data
predictions = model.predict(test_data['sentence'])

# Create a DataFrame with the ID and predicted difficulty
submission = pd.DataFrame({
    'id': test_data['id'],
    'difficulty': predictions
})

# Save the updated DataFrame to a CSV file
file_name = 'sample_submission_1.csv'  # Temporary file name in the Colab environment
submission.to_csv(file_name, index=False)

In [ ]:
# Split the data into features (X) and labels (y)
X = training_data['sentence']
y = training_data['difficulty']

# Encode labels as integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2)

# Fit the classifier to the training data
model.fit(X_train, y_train)

# Predict on the validation data
y_pred = model.predict(X_val)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy pipeline:", accuracy)

Validation Accuracy pipeline: 0.4358706986444213


In [ ]:
# Download necessary NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')

# Load French stop words
french_stop_words = set(stopwords.words('french'))

# Define a function to preprocess the text
def preprocess_text(text):
    # Normalize accents and remove non-ASCII characters
    text = unidecode.unidecode(text)
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenization
    words = word_tokenize(text)
    # Remove stop words
    words = [word for word in words if word not in french_stop_words]
    # Rejoin the words back into one string
    return " ".join(words)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Test the preprocessing function
sample_text = "L'intelligence artificielle est l'étude des agents induits par des machines."
preprocessed_text = preprocess_text(sample_text)
print(preprocessed_text)


lintelligence artificielle letude agents induits machines


In [ ]:
# Apply the preprocessing to the training data
training_data_preprocessed = training_data
training_data_preprocessed['sentence'] = training_data['sentence'].apply(preprocess_text)

# Apply the preprocessing to the test data
test_data_preprocessed = test_data
test_data_preprocessed['sentence'] = test_data['sentence'].apply(preprocess_text)


In [ ]:
# Preprocess the text if necessary (e.g., lowercasing, stemming)
# For simplicity, we'll assume the data is clean and well-formatted for now

# Split the training data into a training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(training_data_preprocessed['sentence'], training_data_preprocessed['difficulty'], test_size=0.2, random_state=42)

# Create a pipeline with a TF-IDF vectorizer and a Logistic Regression classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', LogisticRegression(solver='liblinear')),
])

# Train the model
pipeline.fit(X_train, y_train)

# Predict on the validation set
val_predictions = pipeline.predict(X_val)

# Calculate the accuracy on the validation set
val_accuracy = accuracy_score(y_val, val_predictions)

print(f'Validation Accuracy pipeline with preprocessing: {val_accuracy}')

# Now, let's predict on the actual test data

# Predict the difficulty for the test data
test_predictions = pipeline.predict(test_data_preprocessed['sentence'])

# Create a DataFrame with the ID and predicted difficulty
submission = pd.DataFrame({
    'id': test_data_preprocessed['id'],
    'difficulty': test_predictions
})

# Save the updated DataFrame to a CSV file
file_name = 'sample_submission_2.csv'  # Temporary file name in the Colab environment
submission.to_csv(file_name, index=False)

Validation Accuracy pipeline with preprocessing: 0.4056308654848801


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Split the preprocessed training data into a training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(
    training_data_preprocessed['sentence'],
    training_data_preprocessed['difficulty'],
    test_size=0.2,
    random_state=42
)

# Create a pipeline with a TF-IDF vectorizer and a Support Vector Classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', SVC(kernel='linear'))
])

# We can also use GridSearchCV to find the best parameters for our SVC
param_grid = {
    'classifier__C': [0.1, 1, 10]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')

# Train the model using grid search to find the best parameters
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Predict on the validation set with the best model
val_predictions = best_model.predict(X_val)

# Calculate the accuracy on the validation set
val_accuracy = accuracy_score(y_val, val_predictions)
print(f'Validation Accuracy grid search: {val_accuracy}')

# Predict the difficulty for the preprocessed test data
test_predictions = best_model.predict(test_data_preprocessed['sentence'])

# Create a DataFrame with the ID and predicted difficulty
submission_grid_search = pd.DataFrame({
    'id': test_data['id'],
    'difficulty': test_predictions
})



Validation Accuracy grid search: 0.41605839416058393


In [ ]:
# Assuming grid_search was already done and best_model contains the best estimator

# Retrieve the best parameters from the grid search
best_params = grid_search.best_params_

# Extract the best value of C parameter for SVC
best_C = best_params['classifier__C']

# Now re-create the pipeline with the best parameters and retrain on the entire training data
final_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', SVC(kernel='linear', C=best_C))
])

# Fit the final model on the entire training data set
final_pipeline.fit(training_data_preprocessed['sentence'], training_data_preprocessed['difficulty'])

# Now predict on the test data
final_predictions = final_pipeline.predict(test_data_preprocessed['sentence'])

# Create the final submission DataFrame
final_submission = pd.DataFrame({
    'id': test_data['id'],
    'difficulty': final_predictions
})
# Save the updated DataFrame to a CSV file
file_name = 'sample_submission_GS.csv'  # Temporary file name in the Colab environment
final_submission.to_csv(file_name, index=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
train_data, val_data = train_test_split(
    training_data,
    test_size=0.2,
    random_state=42
)

In [ ]:
# Split the training data
X_train = train_data['sentence']
y_train = train_data['difficulty']

# Create a TF-IDF vectorizer and Naive Bayes classifier pipeline
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Train the model
model.fit(X_train, y_train)

# Predict the difficulty for the test data
predictions = model.predict(val_data['sentence'])

# Create a DataFrame with the ID and predicted difficulty
pred = pd.DataFrame({
    'id': val_data['id'],
    'difficulty': predictions
})

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(val_data['difficulty'], predictions)

print("Validation Accuracy:", accuracy)


Validation Accuracy: 0.410844629822732


In [ ]:
# Split the training data into training and validation sets
train_data_preprocessed, val_data_preprocessed = train_test_split(training_data_preprocessed, test_size=0.2, random_state=42)

In [ ]:
# Split the training data
X_train_preprocessed = train_data_preprocessed['sentence']
y_train = train_data_preprocessed['difficulty']

# Create a TF-IDF vectorizer and Naive Bayes classifier pipeline
model_preprocessed = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Train the model
model_preprocessed.fit(X_train, y_train)

# Predict the difficulty for the test data
predictions_preprocessed = model.predict(val_data['sentence'])

# Create a DataFrame with the ID and predicted difficulty
pred = pd.DataFrame({
    'id': val_data['id'],
    'difficulty': predictions
})


In [ ]:
# Calculate accuracy
accuracy = accuracy_score(val_data_preprocessed['difficulty'], predictions_preprocessed)

print("Validation Accuracy:", accuracy)


Validation Accuracy: 0.410844629822732


In [ ]:
# Split the data into features (X) and labels (y)
X = training_data['sentence']
y = training_data['difficulty']

# Encode labels as integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2)

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform on training data, transform on validation data
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

# Define the decision tree classifier
clf = DecisionTreeClassifier()

# Fit the classifier to the training data
clf.fit(X_train_tfidf, y_train)

# Predict on the validation data
y_pred = clf.predict(X_val_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy decision tree:", accuracy)


Validation Accuracy decision tree: 0.35140771637122004


In [ ]:
# Define the KNN classifier
clf = KNeighborsClassifier(n_neighbors=37)  # You can adjust the number of neighbors

# Fit the classifier to the training data
clf.fit(X_train_tfidf, y_train)

# Predict on the validation data
y_pred = clf.predict(X_val_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy knn:", accuracy)


Validation Accuracy knn: 0.2721584984358707


In [ ]:
# Define the Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust the number of estimators

# Fit the classifier to the training data
clf.fit(X_train_tfidf, y_train)

# Predict on the validation data
y_pred = clf.predict(X_val_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy random forest:", accuracy)


Validation Accuracy random forest: 0.3753910323253389


In [ ]:
# Define the SVM classifier
clf = SVC(kernel='linear', random_state=42)  # You can choose different kernels (linear, polynomial, rbf, etc.)

# Fit the classifier to the training data
clf.fit(X_train_tfidf, y_train)

# Predict on the validation data
y_pred = clf.predict(X_val_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy svc:", accuracy)


Validation Accuracy svc: 0.42127215849843586


In [ ]:
# Define the XGBoost classifier
clf = xgb.XGBClassifier(objective='multi:softmax', random_state=42)  # You can adjust other parameters

# Fit the classifier to the training data
clf.fit(X_train_tfidf, y_train)

# Predict on the validation data
y_pred = clf.predict(X_val_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy gbm:", accuracy)


Validation Accuracy gbm: 0.3983315954118874


In [ ]:
# Define the Multinomial Naive Bayes classifier
clf = MultinomialNB()

# Fit the classifier to the training data
clf.fit(X_train_tfidf, y_train)

# Predict on the validation data
y_pred = clf.predict(X_val_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy naive bayes:", accuracy)


Validation Accuracy naive bayes: 0.4369134515119917


In [ ]:
# Split the data into features (X) and labels (y)
X = training_data['sentence']
y = training_data['difficulty']

# Encode labels as integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the labeled data into training and validation sets
X_train_labeled, X_val, y_train_labeled, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform on labeled training data, transform on validation data
X_train_labeled_tfidf = vectorizer.fit_transform(X_train_labeled)
X_val_tfidf = vectorizer.transform(X_val)

# Define the base classifier (Multinomial Naive Bayes)
base_classifier = MultinomialNB()

# Define the self-training classifier
self_training_classifier = SelfTrainingClassifier(base_classifier)

# Fit the self-training classifier on the labeled training data
self_training_classifier.fit(X_train_labeled_tfidf, y_train_labeled)

# Predict on the validation data
y_pred = self_training_classifier.predict(X_val_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)


Validation Accuracy: 0.410844629822732


/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_self_training.py:212: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


In [ ]:


# Split the data into features (X) and labels (y)
X = training_data['sentence']
y = training_data['difficulty']

# Encode labels as integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform on training data, transform on validation data
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

# Define the base classifier (Decision Tree in this example)
base_classifier = DecisionTreeClassifier(max_depth=1)  # Weak learner

# Define the AdaBoost classifier
clf = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=100, random_state=42)  # You can adjust the number of estimators

# Fit the classifier to the training data
clf.fit(X_train_tfidf, y_train)

# Predict on the validation data
y_pred = clf.predict(X_val_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Validation Accuracy: 0.2940563086548488


In [ ]:
# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', RandomForestClassifier())
])

# Define the hyperparameter space
param_dist = {
    'tfidf__max_features': randint(1000, 5000),  # Number of features to consider
    'tfidf__ngram_range': [(1, 1), (1, 2)],      # Range of n-grams
    'clf__n_estimators': randint(100, 1000),     # Number of trees in the forest
    'clf__max_depth': [None] + list(np.arange(10, 110, 10)),  # Maximum depth of the tree
    'clf__min_samples_split': randint(2, 20),     # Minimum number of samples required to split an internal node
    'clf__min_samples_leaf': randint(1, 20),      # Minimum number of samples required to be at a leaf node
    'clf__bootstrap': [True, False]              # Whether bootstrap samples are used when building trees
}

# Define RandomizedSearchCV
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=50, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

# Fit the model
random_search.fit(X_train, y_train)

# Print the best parameters found
print("Best Parameters:", random_search.best_params_)

# Get the best model
best_model = random_search.best_estimator_

# Evaluate the best model on the validation set
accuracy = best_model.score(X_val, y_val)
print("Validation Accuracy:", accuracy)


Fitting 5 folds for each of 50 candidates, totalling 250 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

def evaluate_model(model, X_val, y_val):
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred, average='weighted')
    recall = recall_score(y_val, y_pred, average='weighted')
    f1 = f1_score(y_val, y_pred, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Example usage with different models:
# Assuming models are trained and X_val_tfidf, y_val are defined

results = []

# List of models to evaluate
models = {
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(solver='liblinear'),
    'Decision Tree': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(kernel='linear', random_state=42),
    'XGBoost': xgb.XGBClassifier(objective='multi:softmax', random_state=42),
    'AdaBoost': AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=100, random_state=42)
}

# Assume vectorizer and training split as in previous example
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

# Evaluate each model
for model_name, model in models.items():
    model.fit(X_train_tfidf, y_train)
    metrics = evaluate_model(model, X_val_tfidf, y_val)
    metrics['model'] = model_name
    results.append(metrics)

# Convert results to DataFrame
results_df = pd.DataFrame(results)
print(results_df)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


   accuracy  precision    recall        f1                model
0  0.410845   0.428029  0.410845  0.410068          Naive Bayes
1  0.405631   0.411018  0.405631  0.404039  Logistic Regression
2  0.351408   0.344254  0.351408  0.345034        Decision Tree
3  0.283629   0.350460  0.283629  0.254637                  KNN
4  0.384776   0.381636  0.384776  0.362797        Random Forest
5  0.416058   0.417252  0.416058  0.415141                  SVM
6  0.387904   0.381500  0.387904  0.379298              XGBoost
7  0.294056   0.343244  0.294056  0.283024             AdaBoost
